# 0.0 - Imports

In [4]:
# Bibliotecas
import inflection
import numpy      as np
import pandas     as pd
import pickle
import seaborn    as sns
import s3fs
import sqlalchemy
import sqlite3
import umap.umap_ as umap
import re
import os

from IPython.display     import HTML, Image
from matplotlib          import pyplot as plt
from pandas_profiling    import ProfileReport
from plotly              import express
from sklearn             import cluster, metrics, preprocessing, decomposition, manifold, ensemble, mixture, neighbors
from scipy.cluster       import hierarchy 
from scipy.stats         import kstest, norm
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

In [5]:
key = os.environ.get('aws_access_key_id')
secret = os.environ.get('aws_secret_access_key')
fs = s3fs.S3FileSystem(anon=False, key=key , secret=secret )

## 0.1 - Helper Functions

In [6]:
# Função para remover camelcase
snakecase = lambda x: inflection.underscore(x)

# Configurações de Layout
def jupyter_settings():
    %matplotlib inline
    #%pylab inline
    
    plt.style.use('ggplot')
    plt.rcParams['figure.figsize'] = (24,9)
    plt.rcParams['font.size'] = 24
    
    display( HTML('<style>.container {width:93% ! important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option('display.expand_frame_repr', False)
    
    sns.set()
    
jupyter_settings()

In [7]:
import os


## 0.2 - Carregamento dos dados

In [8]:
# Carregar os dados para análise.
df = pd.read_csv('../data/raw/Ecommerce.csv', encoding = 'unicode_escape').drop(columns=['Unnamed: 8'])

# 1.0 - Descrição dos Dados

In [9]:
df1 = df.copy()

## 1.1 - Renomear Nome das Colunas

In [10]:
# Armazenando nome antigo das colunas
name_columns = df1.columns.tolist()

# Nome das novas colunas
new_columns = [snakecase(x) for x in name_columns]

# renomear as colunas
df1.columns = new_columns

## 1.2 - FillOut NA

In [11]:
# ---------------- Separação dos dados faltantes e não faltantes ----------------------------------------

# Dados com o customer_id nulo
df_missing  = df1[df1['customer_id'].isna()].copy()

# Dados com o custome_id preenchido
df_not_missing = df1[~df1['customer_id'].isna()].copy()


# --------------- Verificação dos dados que podem coincidir nos dois conjuntos --------------------------

# Listando os invoice_no dos customers na
list_missing_invoice = df_missing['invoice_no'].unique().tolist()

# Verificação
df_not_missing[df_not_missing['customer_id'].isin(list_missing_invoice)]

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country


In [12]:
# ------------------------------------- Preenchimento dos Dados -------------------------------------------   

# Armazenamento dos invoice_no dos customers nulos.
df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())

# Criação dos Customers Id
df_backup['customer_id'] = np.arange(19000, 19000 + len(df_backup), 1)

# Merge do Backup com o dataframe original
df1 = pd.merge(df1, df_backup, on='invoice_no', how='left').copy()

# Junção das colunas customer id em uma única.
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

# remoção das colunas customer_id_x e customer_id_y
df1 = df1.drop(columns=['customer_id_x', 'customer_id_y'])

# Verificação da Remoção dos NA's da coluna Customer.
df1.isna().sum()

invoice_no         0
stock_code         0
description     1454
quantity           0
invoice_date       0
unit_price         0
country            0
customer_id        0
dtype: int64

## 1.3 - Limpeza dos Dados

In [13]:
# Nome dos paíse em tamanho minúsculo.
df1['country'] = df1['country'].apply(lambda x: x.lower())

## 1.4 - Mudança de Variáveis

In [14]:
# Mudança de tipagem do invoice_date
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y')

# customer_id
df1['customer_id'] = df1['customer_id'].apply(lambda x: int(x))

# Verificação das Variáveis
df1.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int64
dtype: object

## 1.5 - Estatística Descritiva 

### 1.5.1 - Atributos Numéricos

In [15]:
# Separando somente os dados numéricos
num_attributes = df1.select_dtypes(include=['int64', 'float64'])


# ------------ Tendências Centrais -----------------------------------

# Média
mean = pd.DataFrame(num_attributes.apply(np.mean)).T

# Mediana
median = pd.DataFrame(num_attributes.apply(np.median)).T


# ------------ Tendências de Dispersão -------------------------------

# Máximo
maximo = pd.DataFrame(num_attributes.apply(np.max)).T

# Mínimo
minimo = pd.DataFrame(num_attributes.apply(np.min)).T

# Intervalo(Range)
intervalo = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T

# Desvio Padrão
desvio = pd.DataFrame(num_attributes.apply(np.std)).T

# Skewness
skew = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T

# Kurtosis
kurtosis = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T


# ---------------------- Construção da Tabela de Estatística Descritiva -------------------------------

# Concatenação das Tabela
tabela_descritiva = pd.concat([minimo, maximo, intervalo, mean, median, desvio, skew, kurtosis]).T.reset_index()
tabela_descritiva.columns = ['atributos', 'minimo', 'maximo', 'intervalo', 'media', 
                             'mediana', 'desvio_padrao', 'skewness', 'kurtosis']

# Exibição da tabela descritiva
tabela_descritiva

,atributos,minimo,maximo,intervalo,media,mediana,desvio_padrao,skewness,kurtosis
0,quantity,-80995.00,80995.0,161990.00,9.552250,3.00,218.080957,-0.264076,119769.160031
1,unit_price,-11062.06,38970.0,50032.06,4.611114,2.08,96.759764,186.506972,59005.719097
2,customer_id,12346.00,22709.0,10363.00,16688.840453,16249.00,2911.408666,0.487449,-0.804287


### 1.5.2 - Atributos Categóricos

In [16]:
# Separação das Variáveis Categóricas
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

# Exibição das variáveis
cat_attributes.columns

Index(['invoice_no', 'stock_code', 'description', 'country'], dtype='object')

#### Invoice No

In [17]:
# Exploração da variável invoice_no 
df_invoice =  df1.loc[cat_attributes['invoice_no'].apply(lambda x: bool(re.search('[^0-9]+', x)))]


# --------- Verificação de quantidades negativas de invoice com letras e negativos -----------------------------------------

# quantidade de dados do df_invoice
print('Quantidade de dados de Invoice_No com a letra C no início: {}'.format(len(df_invoice)))

# Quantidade de quantity negativos no df_invoice
print('Quantidade de dados com o quantity negativo: {}.'.format(len(df_invoice[df_invoice['quantity'] < 0])))

# exibição dos dados
df_invoice.head()

Quantidade de dados de Invoice_No com a letra C no início: 9291
Quantidade de dados com o quantity negativo: 9288.


,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
141,C536379,D,Discount,-1,2016-11-29,27.50,united kingdom,14527
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2016-11-29,4.65,united kingdom,15311
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2016-11-29,1.65,united kingdom,17548
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2016-11-29,0.29,united kingdom,17548
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2016-11-29,0.29,united kingdom,17548


#### StockCode

In [18]:
# Exploração da variável stock_code
df_stock_code =  df1.loc[cat_attributes['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$', x)))]

# Exibição
print(df_stock_code['stock_code'].unique().tolist())
df_stock_code.head()

['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK']


,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
45,536370,POST,POSTAGE,3,2016-11-29,18.00,france,12583
141,C536379,D,Discount,-1,2016-11-29,27.50,united kingdom,14527
386,536403,POST,POSTAGE,1,2016-11-29,15.00,netherlands,12791
1123,536527,POST,POSTAGE,1,2016-11-29,18.00,germany,12662
1814,536544,DOT,DOTCOM POSTAGE,1,2016-11-29,569.77,united kingdom,19001


#### Description

In [19]:
cat_attributes.head()

,invoice_no,stock_code,description,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,united kingdom
1,536365,71053,WHITE METAL LANTERN,united kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,united kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,united kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,united kingdom


#### Country

In [20]:
# Países
cat_attributes['country'].unique()

array(['united kingdom', 'france', 'australia', 'netherlands', 'germany',
       'norway', 'eire', 'switzerland', 'spain', 'poland', 'portugal',
       'italy', 'belgium', 'lithuania', 'japan', 'iceland',
       'channel islands', 'denmark', 'cyprus', 'sweden', 'austria',
       'israel', 'finland', 'bahrain', 'greece', 'hong kong', 'singapore',
       'lebanon', 'united arab emirates', 'saudi arabia',
       'czech republic', 'canada', 'unspecified', 'brazil', 'usa',
       'european community', 'malta', 'rsa'], dtype=object)

# 2.0 - Filtragem dos Dados

In [21]:
df2 = df1.copy()

In [22]:
# Filtragem unit_price
df2 = df2[df2['unit_price'] >=0.04].copy()


# --------------------- Filtragem Atributos Categóricos --------------------

# lista de códigos a serem removido
list_of_codes = df_stock_code['stock_code'].unique().tolist()

# Remoção de códigos do estoque
df2 = df2[~df2['stock_code'].isin(list_of_codes)].copy()

# Remoção da variável description
df2 = df2.drop('description', axis=1).copy()

# Remoção de Países Desnecessários
df2 = df2[~df2['country'].isin(['european community', 'unspecified'])]


# ------------------- Filtragem Atributos Numéricos ------------------------

# Armazenando os dados que foram cancelados ou devolvidos
df_devolucao = df2[df2['quantity'] < 0].copy()

# Armazenando os dados que foram faturados
df_faturamento = df2[df2['quantity'] >= 0].copy()


# ------------------ Remoção de Péssimos Clientes -------------------------

df2 = df2[~df2['customer_id'].isin([16446])].copy()

# 3.0 - Feature Engineering

In [ ]:
df3 = df2.copy()

## 3.1 - Criação das Features


In [ ]:
# Dataframe Referência
df_ref = df_faturamento.drop(['invoice_no', 'stock_code', 'quantity', 
                              'invoice_date','unit_price', 'country'], axis=1).copy()

# Remover duplicidades 
df_ref = df_ref.drop_duplicates().reset_index(drop=True)

### 3.1.1 - Faturamento

In [ ]:
# ----------------------------------- Construção da Feature Faturamento ----------------------------------------

# Criação da variável de total comprado por produto
df_faturamento['total_by_product'] = df_faturamento['quantity'] * df_faturamento['unit_price']

# Obtendo o valor total de compra por cada customer id
df_monetary = df_faturamento[['customer_id', 'total_by_product']].groupby('customer_id').sum().reset_index()

# Renomeando colunas
df_monetary = df_monetary.rename(columns = {'total_by_product' : 'faturamento'})

# Merge do df_monetary com o dataframe referência.
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')

### 3.1.2 - Recência em Dias da Última Compra Realizada

In [ ]:
# -------------------------------------- Construção da Feature Recency -----------------------------------

# Extraindo o último dia de compra do cliente
df_recency = df_faturamento[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()

# Obtendo a quantidade de dias do momento atual da análise até o último dia de compra do cliente
df_recency['recencia_dias'] = (df_recency['invoice_date'].max() - df_recency['invoice_date']).dt.days

# Armazenando somente os dados necessários
df_recency = df_recency[['customer_id', 'recencia_dias']].copy() 

# Merge do df_recency com o dataframe referência.
df_ref = pd.merge(df_ref, df_recency, on='customer_id', how='left')

### 3.1.3 - Quantidade de Tipos de Produtos Comprados

In [ ]:
# ----------------------------------------- Construção da Feature Products Stock_code --------------------------------

# Obtendo a quantidade de tipos de variedades de produtos que cada cliente comprou
df_num_types_products = (df_faturamento[['customer_id', 'stock_code']].groupby('customer_id')
                                                                      .count()
                                                                      .reset_index())

# Renomeando colunas
df_num_types_products = df_num_types_products.rename(columns={'stock_code' : 'qtde_produtos_comprados'})

# Merge do df_frequencily com o dataframe referência.
df_ref = pd.merge(df_ref, df_num_types_products, on='customer_id', how='left')

### 3.1.4 - Taxa de Frequência de Compras Por Dia

In [ ]:
# Criando um DataFrame auxiliar
df_aux = (df_faturamento[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                                       .groupby('customer_id')
                                                                       .agg(max_   = ('invoice_date', 'max'),
                                                                            min_   = ('invoice_date', 'min'),
                                                                            days_  = ('invoice_date', lambda x: ( (x.max() - x.min()).days ) + 1 ),
                                                                            buy_   = ('invoice_no', 'count'))
                                                                       .reset_index())

# Cálculo da taxa de frequência de compra(compras/dia)
df_aux['frequencia_compra'] = df_aux[['days_', 'buy_']].apply(lambda x: x['buy_']/x['days_'], axis=1)

# Tomando os dados da taxa de frequência de compra e os customer_id
df_frequencily = df_aux[['customer_id', 'frequencia_compra']].copy()

# Merge
df_ref = pd.merge(df_ref, df_frequencily, on='customer_id', how='left')

### 3.1.5 - Número de Devoluções

In [ ]:
# Armazenando a quantidade de devoluções por clientes.
df_number_returns = ( df_devolucao[['customer_id', 'quantity']].groupby('customer_id')
                                                               .sum()
                                                               .reset_index()
                                                               .rename(columns={'quantity':'qtde_devolucoes'}) )

# Reajustando valores.
df_number_returns['qtde_devolucoes'] = df_number_returns['qtde_devolucoes']*(-1)

# Merge do dataframe de devoluções com o dataframe de referência
df_ref = pd.merge(df_ref, df_number_returns, on='customer_id', how='left')

# Preenchendo os na's com os valor 0.
df_ref['qtde_devolucoes'] = df_ref['qtde_devolucoes'].apply(lambda x: 0 if pd.isna(x) else x)

# 4.0 - Análise Exploratória dos Dados

In [ ]:
# Variáveis_relevantes
columns_selected = ['customer_id', 
                    'faturamento', 
                    'recencia_dias',
                    'qtde_produtos_comprados', 
                    'frequencia_compra', 
                    'qtde_devolucoes']

df4 = df_ref[columns_selected].copy()

# Dropando os dados na's. No caso, são clientes que realizaram somente uma única compra dentro de uma ano. Logo, faz sentido removê-los, a fim de 
# facilitar a clusterização dos clientes.
df4 = df4.dropna()

In [ ]:
df4 = df4[df4.customer_id.isin([16446]) == False].copy()

# 5.0 - Preparação dos Dados

In [ ]:
df5 = df4.copy()

In [ ]:
mm = preprocessing.MinMaxScaler()

# ------------------------------ Padronização dos dados para input no modelo ------------------------------------------
df5['faturamento'] = mm.fit_transform(df5[['faturamento']])
pickle.dump(mm, open('../src/features/faturamento_scaler.pkl', 'wb')) # Salvando o modelo em pickle na pasta local
pickle.dump(mm, fs.open('s3://insiders-hd/faturamento_scaler.pkl', 'wb')) # Salvando o modelo em pickle na s3 da aws


df5['recencia_dias'] = mm.fit_transform(df5[['recencia_dias']])
pickle.dump(mm, open('../src/features/recencia_dias_scaler.pkl', 'wb')) # Salvando o modelo em pickle na pasta local
pickle.dump(mm, fs.open('s3://insiders-hd/recencia_dias_scaler.pkl', 'wb')) # Salvando o modelo em pickle na s3 da aws


df5['qtde_produtos_comprados'] = mm.fit_transform(df5[['qtde_produtos_comprados']])
pickle.dump(mm, open('../src/features/qtde_produtos_comprados_scaler.pkl', 'wb')) # Salvando o modelo em pickle na pasta local
pickle.dump(mm, fs.open('s3://insiders-hd/qtde_produtos_comprados_scaler.pkl', 'wb')) # Salvando o modelo em pickle na s3 da aws


df5['frequencia_compra'] = mm.fit_transform(df5[['frequencia_compra']])
pickle.dump(mm, open('../src/features/frequencia_compra_scaler.pkl', 'wb')) # Salvando o modelo em pickle na pasta local
pickle.dump(mm, fs.open('s3://insiders-hd/frequencia_compra_scaler.pkl', 'wb')) # Salvando o modelo em pickle na s3 da aws


df5['qtde_devolucoes'] = mm.fit_transform(df5[['qtde_devolucoes']])
pickle.dump(mm, open('../src/features/qtde_devolucoes_scaler.pkl', 'wb')) # Salvando o modelo em pickle na pasta local
pickle.dump(mm, fs.open('s3://insiders-hd/qtde_devolucoes_scaler.pkl', 'wb')) # Salvando o modelo em pickle na s3 da aws

# 6.0 - Seleção das Features

In [ ]:
df6 = df5.copy()

# Dropagem da coluna customer_id
X = df6.drop('customer_id', axis=1)

## 6.1 - Espaço de Embedding UMAP

In [ ]:
# Instanciando o UMAP
reducer = umap.UMAP(random_state=42)
pickle.dump(reducer, open('../src/features/umap_reducer.pkl', 'wb')) # Salvando o modelo na pasta local
pickle.dump(reducer, fs.open('s3://insiders-hd/umap_reducer.pkl', 'wb')) # Salvando o modelo no s3 da aws


# Realizando o mergulho
embedding = reducer.fit_transform(X)

# Separação dos dados para plotagem.

# eixo x e y
df_umap = pd.DataFrame()
df_umap['embedding_x'] = embedding[ : , 0]
df_umap['embedding_y'] = embedding[ : , 1] 

# Plotagem
sns.scatterplot(x='embedding_x', 
                y='embedding_y', 
                data=df_umap);

In [ ]:
# Instanciando o UMAP
#reducer = manifold.TSNE(n_components=2, learning_rate='auto', init='random', n_jobs=-1)

# Realizando o mergulho
#embedding = reducer.fit_transform(X)

# Separação dos dados para plotagem.
# eixo x e y
#df_tsne = pd.DataFrame()
#df_tsne['embedding_x'] = embedding[ : , 0]
#df_tsne['embedding_y'] = embedding[ : , 1] 

# Plotagem
#sns.scatterplot(x='embedding_x', 
#                y='embedding_y', 
#                data=df_tsne);

# 7.0 - Hiperparameter FineTunning

In [ ]:
# Ajuste dos Dados os modelos
df_treino = df6.drop(['customer_id'], axis=1)

# quantidade de cluster
#clusters_feature = [2, 3, 4, 5, 6, 7]

## 7.1 - Hierarchical Clustering 

### Espaço de Embeeding - UMAP

In [ ]:
# Lista Vazia para armazenar os scores
hierarchical_list_embeeding_umap = list()

clusters = np.arange(2,15,1)

for k in clusters:
    # Definição do Modelo e treino
    hierarchical_cluster_model = hierarchy.linkage(df_umap, 'ward')

    # Predição do Modelo
    labels = hierarchy.fcluster(hierarchical_cluster_model, k, criterion='maxclust')

    # Analise da Silhouette 
    sil_score = metrics.silhouette_score(df_umap, labels, metric='euclidean')
    
    # Armazenamento do silhouette score
    hierarchical_list_embeeding_umap.append(sil_score)

# Plotagem dos scores
plt.plot(clusters, hierarchical_list_embeeding_umap, linestyle='--', marker='o', color='b');

# 8.0 - Treinamento do Modelo

## Espaço - UMAP

### Hierarchical CrossValidation

In [ ]:
# Definição do Modelo e Treino
k = 8
hierarchical_cluster_model = hierarchy.linkage(df_umap, 'ward')

# Predição do Modelo
labels_hierarchical_umap = hierarchy.fcluster(hierarchical_cluster_model, k, criterion='maxclust')

# Silhouette Score
print('Silhouette Score: {}'.format(metrics.silhouette_score(df_umap, labels_hierarchical_umap, metric='euclidean')))

# 9.0 - Cluster Analysis

In [ ]:
df9_umap = df_umap.copy()

## Espaço UMAP

### Inspeção por Visualização - Hierarchical

In [ ]:
# Rotulando cada cliente de acordo o o seu cluster
df9_umap['cluster'] = labels_hierarchical_umap

sns.scatterplot(x='embedding_x', y='embedding_y', hue='cluster', palette='deep', data=df9_umap);

## 9.1 - Perfil Final dos Cluster

In [ ]:
# rotular cada cliente de acordo com o seu cluster
df4['cluster'] = labels_hierarchical_umap

# quantidade de clientes por cluster
df_cluster = df4[['customer_id', 'cluster']].groupby('cluster').count().reset_index().rename(columns={'customer_id':'number_customers'})

# Media de Faturamento por Cluster
df_mean_gross_revenue = df4[['faturamento', 
                                'cluster']].groupby('cluster').mean().reset_index().rename(columns= {'faturamento':'media_faturamento'})

# Media de Recência de Compras
df_mean_recencia = df4[['recencia_dias', 
                             'cluster']].groupby('cluster').mean().reset_index().rename(columns={'recencia_dias':'media_recencia'})

# Media de Recências
df_mean_produtos_comprados = df4[['qtde_produtos_comprados', 
                              'cluster']].groupby('cluster').mean().reset_index().rename(columns={'qtde_produtos_comprados':'media_produtos_comprados'})

# Media das compras
df_mean_frequency = df4[['frequencia_compra', 'cluster']].groupby('cluster').mean().reset_index()

# Média de DEvoluções
df_mean_returns = df4[['qtde_devolucoes', 'cluster']].groupby('cluster').mean().reset_index()


# Junção de todos os dados em uma única tabela.
df_aux = pd.merge(df_cluster, df_mean_gross_revenue, how='inner', on='cluster')
df_aux = pd.merge(df_aux, df_mean_recencia, how='inner', on='cluster')
df_aux = pd.merge(df_aux, df_mean_produtos_comprados, how='inner', on='cluster')
df_aux = pd.merge(df_aux, df_mean_frequency, how='inner', on='cluster')
df_aux = pd.merge(df_aux, df_mean_returns, how='inner', on='cluster')

tabela_final = df_aux.copy()
# Criação da coluna que nos dá o percentual de clientes em cada cluster.
tabela_final['number_customer_percentual'] = round((tabela_final['number_customers']/tabela_final['number_customers'].sum())*100, 2)

In [ ]:
tabela_final = tabela_final.sort_values('media_faturamento', ascending=False).reset_index(drop=True)
tabela_final.to_csv('../data/raw/relatorio_final_clusters.csv', index=False)

# 10.0 - Inserção dos Dados no Banco de Dados 

## 10.1 - Criar o Banco de Dados e a Tabela


In [3]:
# Ajuste de Variáveis
df4['qtde_devolucoes'] = df4['qtde_devolucoes'].astype(np.int64) 
df4['cluster'] = df4['cluster'].astype(np.int64)

NameError: name 'df4' is not defined

In [ ]:
# ------------------------- Dropagem do banco da tabela ------------------------------

# Criar o Banco de Dados
connection = sqlite3.connect('insiders_db.sqlite')

# Query para dropar a tabela
query_drop_table = """DROP TABLE insiders"""

# Commando para executar a query
connection.execute(query_drop_table)

# Commit para finalizar a execução
connection.commit()

# Fechando a conexão
connection.close()

In [ ]:
# Criar o Banco de Dados
#connection = sqlite3.connect('insiders_db.sqlite')

# Criar a tabela que vai armazenar os dados
#query_create_table = """
#    CREATE TABLE insiders(
#    customer_id                  INTEGER,
#    faturamento                  REAL,
#    recencia_dias                INTEGER,
#    qtde_produtos_comprados      INTEGER,
#    frequencia_compra            REAL,
#    qtde_devolucoes              INTEGER,
#    cluster                      INTEGER
#    )
#    
#"""
#
## Comando para executar a query que cria a tabela
#connection.execute(query_create_table)
#
## Commit para Finalizar a criação
#connection.commit()
#
## Fechando a conexão
#connection.close()

## 10.2 -  Inserindo os dados na Tabela

In [42]:
# Criar uma conexão com o banco de dados
connection = sqlalchemy.create_engine('sqlite:///insiders_db.sqlite')

# Inserindo os dados
df4.to_sql('insiders', con=connection, if_exists='append', index=False)

5695

## 10.3 - Seleção de Dados para Confirmação de Upload 

In [43]:
# Seleção dos Dados
#dataframe = pd.read_sql("SELECT * FROM insiders", con=connection)